# Deploy SageMaker enpoint and inference from a completed SageMaker training job

## Setup

#### Start SageMaker session

In [2]:
import sagemaker, boto3, json
from sagemaker.session import Session

sagemaker_session = Session()
aws_role = sagemaker_session.get_caller_identity_arn()
aws_region = boto3.Session().region_name
sess = sagemaker.Session()
s3 = boto3.resource('s3', region_name=aws_region)
bucket = s3.Bucket('isicbucket')

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


#### Define model and deployment container info

In [6]:
# Training job name found in SageMaker Studio homepage --> Jobs --> Training
training_job_name = "isic-resnet-v2-finetune-2024-03-12-18-55-50-248"

model_id, model_version = "tensorflow-ic-imagenet-resnet-v2-101-classification-4", "4.0.2"
inference_instance_type = "ml.g4dn.xlarge"

endpoint_name = name_from_base(f"{training_job_name}-deployed-")

#### Retrieve SageMaker inference containers

In [7]:
from sagemaker import image_uris, script_uris
from sagemaker.utils import name_from_base

# Retrieve the inference docker container uri
deploy_image_uri = image_uris.retrieve(
    region=aws_region,
    framework=None,
    image_scope="inference",
    model_id=model_id,
    model_version=model_version,
    instance_type=inference_instance_type,
)
# Retrieve the inference script uri
deploy_source_uri = script_uris.retrieve(
    model_id=model_id, model_version=model_version, script_scope="inference"
)

## Create and deploy SageMaker Estimator

#### Create a SageMaker Estimator object from the training job

In [8]:
from sagemaker.estimator import Estimator

tf_ic_estimator = Estimator.attach(training_job_name=training_job_name,
                                   sagemaker_session=sess)


2024-03-12 19:52:23 Starting - Preparing the instances for training
2024-03-12 19:52:23 Downloading - Downloading the training image
2024-03-12 19:52:23 Training - Training image download completed. Training in progress.
2024-03-12 19:52:23 Uploading - Uploading generated training model
2024-03-12 19:52:23 Completed - Training job completed


#### Use the Estimator object to deploy to a SageMaker endpoint

In [9]:
finetuned_predictor = tf_ic_estimator.deploy(
    initial_instance_count=1,
    instance_type=inference_instance_type,
    entry_point="inference.py",
    image_uri=deploy_image_uri,
    source_dir=deploy_source_uri,
    endpoint_name=endpoint_name,
)

------------!

## Inference the endpoint

#### Local image file

In [10]:
# Image must be read as bytes
with open("ISIC_0000000.jpg", 'rb') as f:
    img = f.read()

query_response = finetuned_predictor.predict(
    img, {"ContentType": "application/x-image", "Accept": "application/json;verbose"}
)
model_predictions = json.loads(query_response)
predicted_label = model_predictions["predicted_label"]
model_predictions

{'probabilities': [0.0349707641,
  0.0313309133,
  0.00636342494,
  0.0517557263,
  0.0590314642,
  0.79188484,
  0.00839714706,
  0.00690865424,
  0.00935700163],
 'labels': ['actinic keratoses',
  'basal cell carcinoma',
  'dermatofibroma',
  'keratosis',
  'melanoma',
  'nevus',
  'squamous cell carcinoma',
  'unknown',
  'vascular skin'],
 'predicted_label': 'nevus'}

#### S3 image file

In [14]:
from SageMaker_utils import read_image_from_s3_as_bytes

img = read_image_from_s3_as_bytes("preprocessed/224x224_center_crop/val/actinic keratoses/ISIC_0024654.jpg", bucket)

query_response = finetuned_predictor.predict(
    img, {"ContentType": "application/x-image", "Accept": "application/json;verbose"}
)
model_predictions = json.loads(query_response)
predicted_label = model_predictions["predicted_label"]
model_predictions

{'probabilities': [0.0586712062,
  0.666958511,
  0.0147649767,
  0.0472798496,
  0.0486128591,
  0.0403140262,
  0.0353702,
  0.0184089672,
  0.0696193948],
 'labels': ['actinic keratoses',
  'basal cell carcinoma',
  'dermatofibroma',
  'keratosis',
  'melanoma',
  'nevus',
  'squamous cell carcinoma',
  'unknown',
  'vascular skin'],
 'predicted_label': 'basal cell carcinoma'}

#### JSON sample request

## Cleanup

In [ ]:
finetuned_predictor.delete_endpoint()